### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
pdx_data = pd.read_csv('pdx_nbhd.csv')

In [3]:
pd.set_option('display.max_rows', None)
pdx_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Data columns (total 92 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            3468 non-null   int64  
 1   listing_url                                   3468 non-null   object 
 2   scrape_id                                     3468 non-null   int64  
 3   last_scraped                                  3468 non-null   object 
 4   source                                        3468 non-null   object 
 5   name                                          3468 non-null   object 
 6   description                                   0 non-null      float64
 7   nearby_attractions                            3468 non-null   object 
 8   picture_url                                   3468 non-null   object 
 9   host_id                                       3468 non-null   i

In [4]:
pdx_data['amenities'].head()

0    []
1    []
2    []
3    []
4    []
Name: amenities, dtype: object

In [5]:
pdx_data = pdx_data.drop(['listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 
                          'nearby_attractions', 'picture_url', 'host_url', 'host_name', 'host_thumbnail_url', 
                          'host_picture_url', 'host_neighbourhood', 'host_verifications', 'neighbourhood_group_cleansed', 
                          'latitude', 'longitude', 'bathrooms', 'bedrooms', 'amenities', 'minimum_minimum_nights', 
                          'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                          'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 
                          'calendar_last_scraped', 'license', 'calculated_host_listings_count', 
                          'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
                          'calculated_host_listings_count_shared_rooms', 'NB_min_nights_low', 'NB_min_nights_high', 
                          'NB_max_nights_low', 'NB_max_nights_high'], axis=1)

In [6]:
pd.set_option('display.max_columns', None)
pdx_data.head()

,id,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighborhood,property_type,room_type,accommodates,bathrooms_text,beds,price,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,NB_listings_count,NB_price_mean,NB_review_location_mean,NB_review_value_mean,NB_avail_30_mean,NB_avail_60_mean,NB_avail_90_mean,NB_avail_365_mean,NB_total_reviews,NB_review_rating_mean,NB_review_accuracy_mean,NB_review_cleanliness_mean,NB_review_checkin_mean,NB_review_communication_mean
0,623976768455730606,4812174,2013-01-23,"Portland, OR",I am a nurse here in Portland at OHSU hospital...,within an hour,100%,98%,t,2,2,t,t,Concordia,Private room in home,Private room,2,1 shared bath,1.0,58.0,2,14,0,0,11,101,33,23,0,2022-07-15,2023-10-22,5.00,5.00,5.00,5.00,4.94,4.88,4.85,f,1.89,135,123.412214,4.932769,4.823000,12.355556,29.162963,47.022222,159.066667,20351,4.900692,4.922462,4.898769,4.952231,4.949923
1,7650094,1197719,2011-09-23,"Portland, OR","Travels\n\nAustralia\n\nEurope: Austria, Franc...",within an hour,100%,100%,f,2,3,t,t,Northwest District,Entire rental unit,Entire home/apt,2,1 bath,1.0,70.0,30,1125,0,19,49,324,12,2,0,2017-08-30,2023-10-28,5.00,5.00,4.92,5.00,5.00,4.92,4.83,f,0.16,185,141.896552,4.911130,4.696836,12.616216,30.086486,48.383784,186.135135,19013,4.809887,4.842994,4.799153,4.891582,4.911695
2,43930130,81209824,2016-07-02,"Portland, OR",Hi Airbnb! I am a third generation General Con...,within an hour,100%,100%,f,3,3,t,t,King,Entire home,Entire home/apt,8,2 baths,4.0,150.0,1,1125,10,36,55,55,78,19,1,2020-08-06,2023-12-17,4.74,4.68,4.68,4.87,4.88,4.83,4.63,f,1.90,125,124.678571,4.874098,4.804262,8.696000,20.640000,35.192000,125.424000,18831,4.867295,4.903361,4.873279,4.954918,4.945328
3,640328648437453436,384749573,2021-01-15,"Wasilla, AK",NaN,within an hour,100%,100%,f,1,1,t,t,Sellwood-Moreland Improvement League,Entire rental unit,Entire home/apt,2,1 bath,1.0,79.0,1,2,0,0,0,0,3,3,3,2023-11-26,2023-12-17,5.00,5.00,5.00,5.00,4.75,5.00,4.75,f,3.00,93,116.413793,4.912529,4.810230,11.118280,26.881720,44.559140,137.978495,11934,4.872184,4.891839,4.904023,4.969195,4.957126
4,802905386849951102,174792040,2018-02-22,NaN,Going someplace new is always exciting. And wh...,within an hour,100%,98%,f,2637,8868,t,t,Portland Downtown,Room in hotel,Private room,2,1 bath,1.0,1040.0,1,1125,30,60,90,360,1,1,0,2023-11-05,2023-11-05,5.00,5.00,5.00,5.00,5.00,5.00,5.00,t,0.65,133,747.592308,4.676140,4.679298,22.368421,47.097744,73.007519,295.233083,2766,4.799123,4.859649,4.831579,4.868421,4.838246


### Clean Dtypes

#### Datetime

In [7]:
pdx_data['host_since'] = pd.to_datetime(pdx_data['host_since'])
pdx_data['first_review'] = pd.to_datetime(pdx_data['first_review'])
pdx_data['last_review'] = pd.to_datetime(pdx_data['last_review'])

In [8]:
pdx_data['host_since'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3468 entries, 0 to 3467
Series name: host_since
Non-Null Count  Dtype         
--------------  -----         
3468 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 27.2 KB


In [9]:
pdx_data.select_dtypes(include = 'datetime64').head(25)

,host_since,first_review,last_review
0,2013-01-23,2022-07-15,2023-10-22
1,2011-09-23,2017-08-30,2023-10-28
2,2016-07-02,2020-08-06,2023-12-17
3,2021-01-15,2023-11-26,2023-12-17
4,2018-02-22,2023-11-05,2023-11-05
5,2010-12-29,2011-02-15,2023-11-05
6,2013-04-23,2013-04-29,2023-12-13
7,2015-04-25,2019-09-15,2019-09-15
8,2018-06-11,2019-08-11,2023-11-19
9,2013-12-04,2021-06-26,2023-11-06


#### To Numeric

In [10]:
pdx_data['bathrooms_text'].head(15)

0        1 shared bath
1               1 bath
2              2 baths
3               1 bath
4               1 bath
5              2 baths
6               1 bath
7              2 baths
8              2 baths
9            2.5 baths
10    1.5 shared baths
11              1 bath
12              1 bath
13              1 bath
14       1 shared bath
Name: bathrooms_text, dtype: object

In [11]:
### set bathrooms_text to two columns; num_bath and is_shared_bathroom

pdx_data['is_shared_bathroom'] = np.where(pdx_data['bathrooms_text'].str.contains('share'),1,0)
pdx_data[['bathrooms_text','is_shared_bathroom']].head()

,bathrooms_text,is_shared_bathroom
0,1 shared bath,1
1,1 bath,0
2,2 baths,0
3,1 bath,0
4,1 bath,0


In [12]:
pdx_data['num_baths'] = pdx_data.bathrooms_text.str.extract(r'(\d+.\d+)').astype('float')
pdx_data[['bathrooms_text','num_baths']].head(15)

,bathrooms_text,num_baths
0,1 shared bath,NaN
1,1 bath,NaN
2,2 baths,NaN
3,1 bath,NaN
4,1 bath,NaN
5,2 baths,NaN
6,1 bath,NaN
7,2 baths,NaN
8,2 baths,NaN
9,2.5 baths,2.5


In [13]:
pdx_data['whole_num_baths'] = pdx_data.bathrooms_text.str.extract('(^\d*)')
pdx_data[['bathrooms_text','num_baths', 'whole_num_baths']].head(15)

,bathrooms_text,num_baths,whole_num_baths
0,1 shared bath,NaN,1
1,1 bath,NaN,1
2,2 baths,NaN,2
3,1 bath,NaN,1
4,1 bath,NaN,1
5,2 baths,NaN,2
6,1 bath,NaN,1
7,2 baths,NaN,2
8,2 baths,NaN,2
9,2.5 baths,2.5,2


In [14]:
pdx_data['num_baths'] = pdx_data['num_baths'].fillna(pdx_data['whole_num_baths'])
pdx_data[['bathrooms_text','num_baths', 'whole_num_baths']].head(15)

,bathrooms_text,num_baths,whole_num_baths
0,1 shared bath,1,1
1,1 bath,1,1
2,2 baths,2,2
3,1 bath,1,1
4,1 bath,1,1
5,2 baths,2,2
6,1 bath,1,1
7,2 baths,2,2
8,2 baths,2,2
9,2.5 baths,2.5,2


In [15]:
pdx_data.drop(columns = ['bathrooms_text','whole_num_baths'], inplace = True)

In [16]:
pdx_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            3468 non-null   int64         
 1   host_id                       3468 non-null   int64         
 2   host_since                    3468 non-null   datetime64[ns]
 3   host_location                 3015 non-null   object        
 4   host_about                    2514 non-null   object        
 5   host_response_time            3053 non-null   object        
 6   host_response_rate            3053 non-null   object        
 7   host_acceptance_rate          3234 non-null   object        
 8   host_is_superhost             3457 non-null   object        
 9   host_listings_count           3468 non-null   int64         
 10  host_total_listings_count     3468 non-null   int64         
 11  host_has_profile_pic          

In [17]:
pdx_data['num_baths'] = pdx_data['num_baths'].apply(pd.to_numeric)

In [18]:
pdx_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            3468 non-null   int64         
 1   host_id                       3468 non-null   int64         
 2   host_since                    3468 non-null   datetime64[ns]
 3   host_location                 3015 non-null   object        
 4   host_about                    2514 non-null   object        
 5   host_response_time            3053 non-null   object        
 6   host_response_rate            3053 non-null   object        
 7   host_acceptance_rate          3234 non-null   object        
 8   host_is_superhost             3457 non-null   object        
 9   host_listings_count           3468 non-null   int64         
 10  host_total_listings_count     3468 non-null   int64         
 11  host_has_profile_pic          

In [19]:
### set host_response_rate, host_acceptance_rate to number

pdx_data[['host_response_rate', 'host_acceptance_rate']].head(25)

,host_response_rate,host_acceptance_rate
0,100%,98%
1,100%,100%
2,100%,100%
3,100%,100%
4,100%,98%
5,100%,100%
6,100%,100%
7,0%,0%
8,100%,100%
9,100%,100%


In [20]:
pdx_data['host_response_rate'] = pdx_data['host_response_rate'].str.replace("%", "")
pdx_data['host_response_rate'] = pd.to_numeric(pdx_data['host_response_rate'])
pdx_data['host_response_rate'] = pdx_data['host_response_rate'] / 100

In [21]:
pdx_data['host_response_rate'].head(25)

0     1.00
1     1.00
2     1.00
3     1.00
4     1.00
5     1.00
6     1.00
7     0.00
8     1.00
9     1.00
10     NaN
11    1.00
12     NaN
13    1.00
14    1.00
15     NaN
16    0.96
17    1.00
18    1.00
19     NaN
20    1.00
21    1.00
22    1.00
23    1.00
24    1.00
Name: host_response_rate, dtype: float64

In [22]:
pdx_data['host_acceptance_rate'] = pdx_data['host_acceptance_rate'].str.replace("%", "")
pdx_data['host_acceptance_rate'] = pd.to_numeric(pdx_data['host_acceptance_rate'])
pdx_data['host_acceptance_rate'] = pdx_data['host_acceptance_rate'] / 100

In [23]:
pdx_data['host_acceptance_rate'].head(25)

0     0.98
1     1.00
2     1.00
3     1.00
4     0.98
5     1.00
6     1.00
7     0.00
8     1.00
9     1.00
10     NaN
11    1.00
12    1.00
13    1.00
14    0.99
15     NaN
16    0.65
17    0.89
18    0.98
19    1.00
20    1.00
21    1.00
22    0.99
23    0.87
24    0.91
Name: host_acceptance_rate, dtype: float64

#### Boolean numeric

In [24]:
### Host_about isnull = 0, else 1

pdx_data['host_about'].head()

0    I am a nurse here in Portland at OHSU hospital...
1    Travels\n\nAustralia\n\nEurope: Austria, Franc...
2    Hi Airbnb! I am a third generation General Con...
3                                                  NaN
4    Going someplace new is always exciting. And wh...
Name: host_about, dtype: object

In [25]:
pdx_data['has_host_about'] = np.where(pdx_data['host_about'].isna(), 0, 1)
pdx_data[['has_host_about','host_about']].head()

,has_host_about,host_about
0,1,I am a nurse here in Portland at OHSU hospital...
1,1,"Travels\n\nAustralia\n\nEurope: Austria, Franc..."
2,1,Hi Airbnb! I am a third generation General Con...
3,0,NaN
4,1,Going someplace new is always exciting. And wh...


In [26]:
pdx_data.drop(columns = ['host_about'], inplace = True)

In [27]:
# set host location to host_loc_is_portland

pdx_data['host_location'].sort_values().unique()

array(['Allentown, PA', 'Anaheim, CA', 'Ashland, OR', 'Atlanta, GA',
       'Auburn, WA', 'Austin, TX', 'Beaverton, OR', 'Bellevue, WA',
       'Bend, OR', 'Berkeley, CA', 'Beverly Hills, CA', 'Birmingham, AL',
       'Boise, ID', 'California, United States', 'Camas, WA', 'Canby, OR',
       'Cathedral City, CA', 'Chandler, AZ', 'Chicago, IL', 'Chico, CA',
       'Clackamas, OR', 'Colorado, United States', 'Corbett, OR',
       'Coronado, CA', 'Corvallis, OR', 'Denver, CO', 'Detroit, MI',
       'Dover, NH', 'Edmonds, WA', 'El Cerrito, CA', 'Eugene, OR',
       'Florida, United States', 'Geelong West, Australia',
       'Georgetown, TX', 'Glenwood, OR', 'Gresham, OR',
       'Hamilton, Canada', 'Hanalei, HI', 'Happy Valley, OR',
       'Hillsboro, OR', 'Honolulu, HI', 'Houston, TX', 'Irvine, CA',
       'Jersey City, NJ', 'Joshua Tree, CA', 'Kailua-Kona, HI',
       'Kansas City, MO', 'Kapaʻa, HI', 'Keizer, OR', 'Kihei, HI',
       'Lafayette, OR', 'Lake Forest, CA', 'Lake Oswego, OR',

In [28]:
pdx_data['host_loc_is_portland'] = np.where(pdx_data['host_location'] == 'Portland, OR', 1, 0)
pdx_data[['host_loc_is_portland','host_location']].head()

,host_loc_is_portland,host_location
0,1,"Portland, OR"
1,1,"Portland, OR"
2,1,"Portland, OR"
3,0,"Wasilla, AK"
4,0,NaN


In [29]:
pdx_data.drop(columns = ['host_location'], inplace = True)

In [30]:
# set host_is_superhost, host_has_profile_pic, host_identity_verified, instant_bookable, 1 or 0
pdx_data[['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']].head()

,host_is_superhost,host_has_profile_pic,host_identity_verified,instant_bookable
0,t,t,t,f
1,f,t,t,f
2,f,t,t,f
3,f,t,t,f
4,f,t,t,t


In [31]:
pdx_data['host_is_superhost'].replace({'t' : 1, 'f' : 0},inplace = True)
pdx_data['host_has_profile_pic'].replace({'t' : 1, 'f' : 0},inplace = True)
pdx_data['host_identity_verified'].replace({'t' : 1, 'f' : 0},inplace = True)
pdx_data['instant_bookable'].replace({'t' : 1, 'f' : 0},inplace = True)

pdx_data[['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']].head()

,host_is_superhost,host_has_profile_pic,host_identity_verified,instant_bookable
0,1.0,1,1,0
1,0.0,1,1,0
2,0.0,1,1,0
3,0.0,1,1,0
4,0.0,1,1,1


### Categorical Data

In [32]:
# Get dummies for neighborhood, host response time, property type, room type

In [33]:
def shape_ratio(df):
    shape = df.shape
    rows = shape[0]
    cols = shape[1]
    print(cols/rows)

In [34]:
# ideally i want ratio under .33 (fewer than 1/3rd as many columns as rows)

shape_ratio(pdx_data)
    
# check current ratio

0.015859284890426758


In [35]:
nb_df = pd.get_dummies(pdx_data['neighborhood'], dtype=float, prefix = 'nb_')

In [36]:
hrt_df = pd.get_dummies(pdx_data['host_response_time'], dtype=float, prefix = 'hrt_')

In [37]:
pt_df = pd.get_dummies(pdx_data['property_type'], dtype=float, prefix = 'pt_')

In [38]:
rt_df = pd.get_dummies(pdx_data['room_type'], dtype=float, prefix = 'rt_')

In [39]:
pdx_data.drop(columns = ['neighborhood', 'host_response_time', 'property_type', 'room_type'], inplace = True)

In [40]:
pdx_data.shape

(3468, 51)

In [41]:
pdx_data = pdx_data.merge(nb_df, left_index = True, right_index = True)

In [42]:
pdx_data = pdx_data.merge(hrt_df, left_index = True, right_index = True)

In [43]:
pdx_data = pdx_data.merge(pt_df, left_index = True, right_index = True)

In [44]:
pdx_data = pdx_data.merge(rt_df, left_index = True, right_index = True)

In [45]:
pdx_data.shape

(3468, 193)

In [46]:
shape_ratio(pdx_data)

0.055651672433679356


In [47]:
pdx_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Columns: 193 entries, id to rt__Shared room
dtypes: datetime64[ns](3), float64(168), int32(3), int64(19)
memory usage: 5.1 MB


### Deal with NaNs

In [48]:
### How much data is lost if we drop them all?

pdx_prep = pdx_data.dropna()

In [49]:
pdx_prep.shape

(2712, 193)

In [50]:
shape_ratio(pdx_prep)

0.07116519174041298


shape ratio is below desired threshold of .33, dataset is still larget enough even with NAs dropped.

In [51]:
pdx_prep.to_csv('pdx_prep.csv', index=False)  

### Train test split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(pdx_prep.drop(columns='price'), 
                                                    pdx_prep.price, test_size=0.3, 
                                                    random_state=47)

# removed price as our target feature

In [53]:
date_list = ['host_since', 'first_review', 'last_review']
date_train = X_train[date_list]
date_test = X_test[date_list]
X_train.drop(columns=date_list, inplace=True)
X_test.drop(columns=date_list, inplace=True)
X_train.shape, X_test.shape

# removed datetime columns since they are not numeric

((1898, 189), (814, 189))

In [54]:
X_train.dtypes

# sanity check

id                                            int64
host_id                                       int64
host_response_rate                          float64
host_acceptance_rate                        float64
host_is_superhost                           float64
host_listings_count                           int64
host_total_listings_count                     int64
host_has_profile_pic                          int64
host_identity_verified                        int64
accommodates                                  int64
beds                                        float64
minimum_nights                                int64
maximum_nights                                int64
availability_30                               int64
availability_60                               int64
availability_90                               int64
availability_365                              int64
number_of_reviews                             int64
number_of_reviews_ltm                         int64
number_of_re

In [55]:
X_test.dtypes

# sanity check

id                                            int64
host_id                                       int64
host_response_rate                          float64
host_acceptance_rate                        float64
host_is_superhost                           float64
host_listings_count                           int64
host_total_listings_count                     int64
host_has_profile_pic                          int64
host_identity_verified                        int64
accommodates                                  int64
beds                                        float64
minimum_nights                                int64
maximum_nights                                int64
availability_30                               int64
availability_60                               int64
availability_90                               int64
availability_365                              int64
number_of_reviews                             int64
number_of_reviews_ltm                         int64
number_of_re

### Standardize numeric columns

In [56]:
scaler = StandardScaler()

In [57]:
scaler.fit(X_train)
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)